In [20]:
import pandas as pd
import numpy as np
from os import listdir
from configuration import get_config
from association import compute_iou

IMG_PATH = r"C:/javier/personal_projects/computer_vision/data/KITTI_object_tracking/images/training/image_02/0000/"
DET_FILE = r"C:/javier/personal_projects/computer_vision/data/KITTI_object_tracking/detections_regionlet/training/det_02/0000.txt"
DATASET = 'KITTI'

In [33]:
# load dets
config = get_config(DATASET)
dets = pd.read_csv(DET_FILE, header=None, sep=" ")
dets.columns = config['det_cols']

# filter dets with low score
min_score = -1.0
dets = dets[dets['score'] > min_score]

In [ ]:
# sort by score
dets = dets.sort_values(by='score', ascending=False).reset_index(drop=True)

iou_thr = 0.2
picked_dets_indexes = []
frames = dets['frame'].unique()
for i, frame in enumerate(frames):
    suppress_index = []
    cur_dets = dets.loc[dets['frame'] == frame, :]
    cur_indexes = list(cur_dets.index)
    for j in cur_indexes:   # already sorted by score
        if j in suppress_index:
            continue
        picked_dets_indexes.append(j)
        box_ref = np.array([cur_dets.loc[j, 'top'], cur_dets.loc[j, 'left'], cur_dets.loc[j, 'bottom'], cur_dets.loc[j, 'right']])
        for k in cur_indexes[j+1:]:
            box_test = np.array([cur_dets.loc[k, 'top'], cur_dets.loc[k, 'left'], cur_dets.loc[k, 'bottom'], cur_dets.loc[k, 'right']])
            iou = compute_iou(box_ref, box_test)
            if iou > iou_thr:
                suppress_index.append(k)

nms_dets = dets.loc[picked_dets_indexes, :]

nms_dets.to_csv('0000_nms_dets.csv', sep=" ", index=False, header=False)

In [23]:
import numpy as np
from scipy.optimize import linear_sum_assignment

cost_matrix = np.array([
    [0, 2, 3],
    [2, 1, 0],
    [-100, -10, -20]
])
row_ind, col_ind = linear_sum_assignment(cost_matrix, maximize=True)
row_ind, col_ind 

(array([0, 1, 2], dtype=int64), array([2, 0, 1], dtype=int64))